<img src="Bilder/ost_logo.png" width="240"  align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN10/ANN10_WetterJena_SOLUTION_pl.ipynb)

# Wettervorhersage mit Hilfe eines LSTM-Modells

**Authors:** [Prabhanshu Attri](https://prabhanshu.com/github), [Yashika Sharma](https://github.com/yashika51), [Kristi Takach](https://github.com/ktakattack), [Falak Shah](https://github.com/falaktheoptimist), [Christoph Würsch](christoph.wuersch@ost.ch)


**Beschreibung:** Dieses Notebook demonstriert, wie man Zeitreihenvorhersagen mit Hilfe eines LSTM-Modells durchführt.

## Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


## Jenaer Klima-Datensatz

Wir werden den Jenaer Klimadatensatz verwenden, der vom [Max-Planck-Institut für Biogeochemie](https://www.bgc-jena.mpg.de/wetter/) zur Verfügung gestellt wird. Der Datensatz besteht aus 14 Merkmalen wie Temperatur, Druck, Feuchtigkeit usw., die einmal pro 10 Minuten aufgezeichnet werden.

- [Hier](https://www.bgc-jena.mpg.de/wetter/Wetterstation.pdf) finden Sie eine Beschreibung der Wetterstation und der damit aufgezeichneten Grössen.
- **Standort**: Wetterstation, Max-Planck-Institut für Biogeochemie in Jena, Deutschland
- **Betrachteter Zeitrahmen**: 10. Januar 2009 - 31. Dezember 2016


Die folgende Tabelle zeigt die Spaltennamen, ihre Werteformate und ihre Beschreibung.


Index| Features      |Format             |Description              |
----:|:--------------|:-----------------:|:------------------------|
1    |Date Time      |01.01.2009 00:10:00|Date-time referenz       |
2    |p (mbar)       |996.52             |Die von Pascal abgeleitete SI-Einheit des Drucks, die zur Quantifizierung des Innendrucks verwendet wird. In meteorologischen Berichten wird der atmosphärische Druck in der Regel in Millibar angegeben.|
3    |T (degC)       |-8.02              |Temperatur in Celsius |
4    |Tpot (K)       |265.4              |Temperatur in Kelvin |
5    |Tdew (degC)    |-8.9               |Temperatur in Celsius relativ zur Luftfeuchtigkeit. Der Taupunkt ist ein Mass für die absolute Menge an Wasser in der Luft. Der DP ist die Temperatur, bei der die Luft nicht mehr die gesamte Feuchtigkeit halten kann und Wasser kondensiert. |
6    |rh (%)         |93.3               |Relative Luftfeuchtigkeit ist ein Mass dafür, wie gesättigt die Luft mit Wasserdampf ist; der %RH-Wert bestimmt die in Sammelobjekten enthaltene Wassermenge. |
7    |VPmax (mbar)   |3.33               |Sättigungsdampfdruck         |
8    |VPact (mbar)   |3.11               |Dampfdruck                   |
9    |VPdef (mbar)   |0.22               |Dampfdruckdefizit            |
10   |sh (g/kg)      |1.94               |Spezifische Feuchtigkeit     |
11   |H2OC (mmol/mol)|3.12               |Wasserdampfkonzentration     |
12   |rho (g/m ** 3) |1307.75            |Luftdichte                   |
13   |wv (m/s)       |1.03               |Windgeschwindigkeit          |
14   |max. wv (m/s)  |1.75               |Maximale Windgeschwindigkeit |
15   |wd (deg)       |152.3              |Windrichtung in Grad         |



In [ ]:
import os
import requests
from zipfile import ZipFile
import pandas as pd
import numpy as np

np.Inf = np.inf  # Nur als Hotfix, nicht langfristig gedacht

# Zielpfade
DATA_DIR = "data/jena"
ZIP_FILENAME = "jena_climate_2009_2016.csv.zip"
CSV_FILENAME = "jena_climate_2009_2016.csv"
ZIP_PATH = os.path.join(DATA_DIR, ZIP_FILENAME)
CSV_PATH = os.path.join(DATA_DIR, CSV_FILENAME)

# URL
URL = "https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip"

# Daten herunterladen, falls nicht vorhanden
os.makedirs(DATA_DIR, exist_ok=True)
if not os.path.exists(CSV_PATH):
    print("🔽 Lade Jena Climate ZIP herunter...")
    response = requests.get(URL)
    with open(ZIP_PATH, "wb") as f:
        f.write(response.content)

    print("📦 Entpacke ZIP...")
    with ZipFile(ZIP_PATH, "r") as zip_ref:
        zip_ref.extractall(DATA_DIR)

    print("✅ Datei bereit:", CSV_PATH)
else:
    print("✅ Datei bereits vorhanden:", CSV_PATH)

# CSV laden
df = pd.read_csv(CSV_PATH)
df


## Visualisierung der Rohdaten

Um uns ein Gefühl für die Daten zu vermitteln, mit denen wir arbeiten, wurde jedes Merkmal unten grafisch dargestellt.
Dies zeigt das eindeutige Muster jedes Merkmals über den Zeitraum von 2009 bis 2016.
Sie zeigt auch, wo Anomalien vorhanden sind, die während der Normalisierung behandelt werden.

In [ ]:
titles = [
    "Pressure",
    "Temperature",
    "Temperature in Kelvin",
    "Temperature (dew point)",
    "Relative Humidity",
    "Saturation vapor pressure",
    "Vapor pressure",
    "Vapor pressure deficit",
    "Specific humidity",
    "Water vapor concentration",
    "Airtight",
    "Wind speed",
    "Maximum wind speed",
    "Wind direction in degrees",
]

feature_keys = [
    "p (mbar)",
    "T (degC)",
    "Tpot (K)",
    "Tdew (degC)",
    "rh (%)",
    "VPmax (mbar)",
    "VPact (mbar)",
    "VPdef (mbar)",
    "sh (g/kg)",
    "H2OC (mmol/mol)",
    "rho (g/m**3)",
    "wv (m/s)",
    "max. wv (m/s)",
    "wd (deg)",
]

colors = [
    "blue",
    "orange",
    "green",
    "red",
    "purple",
    "brown",
    "pink",
    "gray",
    "olive",
    "cyan",
]

date_time_key = "Date Time"


def show_raw_visualization(data):
    time_data = data[date_time_key]
    fig, axes = plt.subplots(
        nrows=7, ncols=2, figsize=(15, 20), dpi=80, facecolor="w", edgecolor="k"
    )
    for i in range(len(feature_keys)):
        key = feature_keys[i]
        c = colors[i % (len(colors))]
        t_data = data[key]
        t_data.index = time_data
        t_data.head()
        ax = t_data.plot(
            ax=axes[i // 2, i % 2],
            color=c,
            title="{} - {}".format(titles[i], key),
            rot=25,
        )
        ax.legend([titles[i]])
    plt.tight_layout()


show_raw_visualization(df)


Diese Heatmap zeigt die Korrelation zwischen verschiedenen Merkmalen.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


def show_heatmap(data, figsize=(12, 10), fontsize=12, cmap="coolwarm"):
    corr = data.corr()
    plt.figure(figsize=figsize)
    sns.heatmap(
        corr,
        annot=False,
        fmt=".2f",
        cmap=cmap,
        xticklabels=True,
        yticklabels=True,
        cbar_kws={"shrink": 0.8, "label": "Correlation"},
    )
    plt.xticks(rotation=45, ha="right", fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    plt.title("Feature Correlation Heatmap", fontsize=fontsize + 2)
    plt.tight_layout()
    plt.show()


# 'Date Time' entfernen und Heatmap anzeigen
numeric_df = df.drop(columns=["Date Time"])
show_heatmap(numeric_df)


## Vorverarbeitung der Daten

- Hier wählen wir ~300.000 Datenpunkte für das Training aus. 
- Die Beobachtung wird alle 10 Minuten aufgezeichnet, d.h. 6 Mal pro Stunde. Wir werden einen Punkt pro Stunde neu auswählen, da keine drastische Veränderung innerhalb von 60 Minuten erwartet wird. Wir tun dies über das Argument `sampling_rate` Argument im Dienstprogramm `timeseries_dataset_from_array`.

- Wir verfolgen die Daten der letzten 720 Zeitstempel (720/6=120 Stunden). Diese Daten werden werden zur Vorhersage der Temperatur nach 72 Zeitstempeln (72/6=12 Stunden) verwendet.

- Da jedes Merkmal über Werte mit unterschiedlichen Bereichen hat, führen wir eine Normalisierung durch, um die Merkmalswerte auf einen Bereich von `[0, 1]` zu beschränken, bevor ein neuronales Netz zu trainieren. Dazu subtrahieren wir den Mittelwert und dividieren ihn durch die Standardabweichung jedes Merkmals.

- 71.5 % der Daten werden zum Trainieren des Modells verwendet, d. h. 300.693 Zeilen. `Split_fraction` kann geändert werden, um diesen Prozentsatz zu ändern.

Dem Modell werden die Daten der ersten 5 Tage als Input gezeigt, d. h. 720 Beobachtungen, die stündlich abgetastet werden.
Die Temperatur nach 72 (12 Stunden $\times$ 6 Beobachtungen pro Stunde) wird als Label verwendet.


In [ ]:
split_fraction = 0.715
train_split = int(split_fraction * int(df.shape[0]))
step = 6

past = 720
future = 72
learning_rate = 0.001
batch_size = 256
epochs = 10


def normalize(data, train_split):
    data_mean = data[:train_split].mean(axis=0)
    data_std = data[:train_split].std(axis=0)
    return (data - data_mean) / data_std


Aus der Korrelationskarte ist ersichtlich, dass einige Parameter wie die relative Luftfeuchtigkeit und die
Spezifische Luftfeuchtigkeit *redundant* sind. Daher werden wir nur ausgewählte Merkmale verwenden, nicht alle.

In [ ]:
print(
    "The selected parameters are:",
    ", ".join([titles[i] for i in [0, 1, 5, 7, 8, 10, 11]]),
)
selected_features = [feature_keys[i] for i in [0, 1, 5, 7, 8, 10, 11]]
features = df[selected_features]
features.index = df[date_time_key]
features.head()

features = normalize(features.values, train_split)
features = pd.DataFrame(features)
features.head()

train_data = features.loc[0 : train_split - 1]
val_data = features.loc[train_split:]


# Trainingsdatensatz

Die Label des Trainingsdatensatzes beginnen mit der 792. Beobachtung (720 + 72).

In [ ]:
start = past + future
end = start + train_split

x_train = train_data[[i for i in range(7)]].values
y_train = features.iloc[start:end][[1]]

sequence_length = int(past / step)


Die Funktion `Timeseries_dataset_from_array` nimmt eine Folge von Datenpunkten auf, die in
gleichen Intervallen gesammelten Datenpunkte, zusammen mit Zeitreihenparametern wie Länge der
Sequenzen/Fenster, Abstände zwischen zwei Sequenzen/Fenstern usw., um Stapel von
Teilzeitreihen-Eingänge und -Ziele, die aus der Hauptzeitreihe entnommen werden.

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader


class TimeseriesDataset(Dataset):
    def __init__(self, x_data, y_data, sequence_length, sampling_rate):
        self.x_data = x_data
        self.y_data = y_data
        self.sequence_length = sequence_length
        self.sampling_rate = sampling_rate

        # Anzahl der verwendbaren Startpositionen
        self.indices = list(range(0, len(x_data) - sequence_length * sampling_rate + 1))

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        i = self.indices[idx]
        x_seq = self.x_data[
            i : i + self.sequence_length * self.sampling_rate : self.sampling_rate
        ]
        y = self.y_data[i + self.sequence_length * self.sampling_rate - 1]
        return torch.tensor(x_seq, dtype=torch.float32), torch.tensor(
            y, dtype=torch.float32
        )


# Dataset initialisieren
train_dataset = TimeseriesDataset(
    x_data=x_train,
    y_data=y_train.values,
    sequence_length=sequence_length,
    sampling_rate=step,
)

# DataLoader erzeugen
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


## Validierungsdatensatz

Der Validierungsdatensatz darf die letzten 792 Zeilen nicht enthalten, da wir für diese Datensätze keine Beschriftungsdaten haben. Daher muss 792 vom Ende der Daten subtrahiert werden.

Der Validierungsdatensatz muss mit 792 Zeilen nach `train_split beginnen`, daher müssen wir Folgendes hinzufügen
`past+future` (792) zu `label_start` addieren.

In [ ]:
x_end = len(val_data) - past - future

label_start = train_split + past + future

x_val = val_data.iloc[:x_end][[i for i in range(7)]].values
y_val = features.iloc[label_start:][[1]]

# Dataset für Validation
val_dataset = TimeseriesDataset(
    x_data=x_val,
    y_data=y_val.values,
    sequence_length=sequence_length,
    sampling_rate=step,
)

# DataLoader für Validation
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


# Einen Batch aus dem Training holen
for batch in train_loader:
    inputs, targets = batch
    break  # nur den ersten Batch

print("Input shape:", inputs.shape)  # z.B. (256, 120, 7)
print("Target shape:", targets.shape)  # z.B. (256, 1)


## (a) Modell-Architektur erstellen und kompilieren

In [ ]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F


class LSTMRegressor(pl.LightningModule):
    def __init__(self, input_size, hidden_size=128, learning_rate=0.001):
        super().__init__()
        self.save_hyperparameters()

        self.lstm = nn.LSTM(
            input_size=input_size, hidden_size=hidden_size, batch_first=True
        )
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate

    def forward(self, x):
        # x: (batch, seq_len, features)
        lstm_out, _ = self.lstm(x)
        last_output = lstm_out[:, -1, :]  # nur der letzte Zeitschritt
        return self.fc(last_output)

    def training_step(self, batch, batch_idx):
        x, y = batch
        preds = self(x).squeeze()
        loss = F.mse_loss(preds, y.squeeze())
        self.log(
            "train_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True
        )
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        preds = self(x).squeeze()
        loss = F.mse_loss(preds, y.squeeze())
        self.log(
            "val_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True
        )
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)


## (b) Modell trainieren

In [ ]:
model = LSTMRegressor(
    input_size=x_train.shape[1],  # z.B. 7
    hidden_size=32,
    learning_rate=learning_rate,
)


In [ ]:
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import EarlyStopping

# Logger erstellen
csv_logger = CSVLogger("logs", name="lstm_weather_forecast")

# EarlyStopping Callback erstellen
early_stopping_callback = EarlyStopping(
    monitor="val_loss",  # Die Metrik, die überwacht werden soll (Validierungsverlust)
    patience=4,  # Anzahl der Epochen ohne Verbesserung, nach denen das Training gestoppt wird
    mode="min",  # Wir wollen den Validierungsverlust minimieren
)

# Trainer mit CSVLogger und EarlyStopping
trainer = pl.Trainer(
    max_epochs=epochs,
    accelerator="auto",
    devices=1,
    logger=csv_logger,
    callbacks=[early_stopping_callback],  # Füge den Callback hier hinzu
)

trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)


## (c) Lernkurven visualisieren

Wir können den Verlust mit der nachstehenden Funktion veranschaulichen. Nach einem Punkt hört der Verlust auf abnehmend.

In [ ]:
import os
import pandas as pd


def find_latest_metrics_path(base_log_dir):
    version_dirs = [d for d in os.listdir(base_log_dir) if d.startswith("version_")]
    if not version_dirs:
        raise FileNotFoundError("Keine version_x Ordner im Log-Verzeichnis gefunden.")

    # Nach Versionsnummer sortieren
    version_dirs.sort(key=lambda x: int(x.split("_")[1]))
    latest_version = version_dirs[-1]

    return os.path.join(base_log_dir, latest_version, "metrics.csv")


# Hauptverzeichnis für Logs
base_log_dir = os.path.join("logs", "lstm_weather_forecast")
metrics_path = find_latest_metrics_path(base_log_dir)

print("Neueste metrics.csv gefunden unter:", metrics_path)

# DataFrame vorbereiten
df = pd.read_csv(metrics_path)
df_train = df[["epoch", "train_loss"]].dropna().copy()
df_val = df[["epoch", "val_loss"]].dropna().copy()

# Merge auf 'epoch'
df_merged = pd.merge(df_train, df_val, on="epoch", suffixes=("_train", "_val"))

df_merged



In [ ]:
# plotte df_merged mit seaborn
import seaborn as sns
import matplotlib.pyplot as plt

# Setze den Stil für die Plots
sns.set(style="whitegrid")
# Erstelle eine Figur
plt.figure(figsize=(10, 6))
# Plot für den Verlust
sns.lineplot(
    data=df_merged, x="epoch", y="train_loss", label="Trainingsverlust"
)
sns.lineplot(
    data=df_merged, x="epoch", y="val_loss", label="Validierungsverlust"
)
plt.title("Trainings- und Validierungsverlust")
plt.xlabel("Epoch")
plt.ylabel("Verlust")
plt.legend()
plt.tight_layout()
plt.show()


## (d) Wettervorhersage

Das oben trainierte Modell ist nun in der Lage, Vorhersagen für 5 Wertesätze aus Validierungssatz.

In [ ]:
import matplotlib.pyplot as plt
import torch


def show_plot(plot_data, delta, title):
    labels = ["History", "True Future", "Model Prediction"]
    marker = [".-", "rx", "go"]
    time_steps = list(range(-(plot_data[0].shape[0]), 0))
    if delta:
        future = delta
    else:
        future = 0

    plt.title(title)
    for i, val in enumerate(plot_data):
        if i:
            plt.plot(future, val, marker[i], markersize=10, label=labels[i])
        else:
            plt.plot(time_steps, val.flatten(), marker[i], label=labels[i])
    plt.legend()
    plt.xlim([time_steps[0], (future + 5) * 2])
    plt.xlabel("Time-Step")
    plt.show()


# Vorhersagen für 5 Beispiele aus dem Validierungs-Dataloader
model.eval()  # Modell in den Evaluierungsmodus setzen
with torch.no_grad():  # Keine Gradientenberechnung
    for i, (x, y) in enumerate(val_loader):
        if i >= 5:  # Nur 5 Beispiele anzeigen
            break
        predictions = model(x).squeeze().numpy()  # Vorhersagen des Modells
        show_plot(
            [x[0][:, 1].numpy(), y[0].numpy(), predictions[0]],
            delta=12,
            title="Single Step Prediction",
        )
